In [3]:
import os
import pandas as pd
import numpy as np
import random
from math import ceil
from time import time
import torch
from lightgbm import LGBMClassifier, Dataset

In [4]:
import matplotlib.pyplot as plt

In [5]:
%matplotlib notebook

In [6]:
dataDir= '../../data'

In [7]:
user_files = [ 'member_info_1205.csv', 'member_info_1205_category_count_attr.csv', 'member_info_user_invite_count.csv']
user_df_ls = []
for file in user_files:
     user_df_ls.append(pd.read_csv(os.path.join(dataDir, file),
                      # usecols= ['user_id', 'gender', 'visit_freq', 'binary_A',
                      #        'binary_B', 'binary_C', 'binary_D', 'binary_E',
                      #        'category_A',
                      #        'category_B',
                      #        'category_C',
                      #        'category_D', 'category_E', 'salt_value', 'follow_topics_mp', 'interest_topics'],
                      index_col='user_id',
                      sep='\t',
                      # nrows= 10000,
                      ))
user_df = pd.concat(user_df_ls, axis = 1)

In [8]:
print('loading data...')
quest_df = pd.read_csv(os.path.join(dataDir, 'question_info_1111.csv'),
                       # usecols= ['question_id', 'title_SW', 'title_W', 'question_topics_mp', 'title_W_ind', 'create_day',
                       #           'has_describe', 'describe_length'],
                       index_col='question_id',
                       sep='\t',
                       # nrows= 10000
                       )

loading data...


In [9]:
quest_df.rename(columns = {'create_day': 'question_create_day', 'invite_count': 'question_invite_count'}, inplace= True)
user_df.rename(columns = {'invite_count': 'user_invite_count'}, inplace= True)

In [41]:
train_invite_files = ['invite_info_[3840, 3860]-[3861, 3867].csv', 
                      'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_ans_info.csv',
                      'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_invite_describe.csv',
                      'invite_info_[3840, 3860]-[3861, 3867]_topics_itsc_count.csv',
                       'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_topics_dist_describe.csv',
                      'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_meanpool_dist.csv', 
                       'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_minpool_dist.csv',
                     'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_maxpool_dist.csv',
                       'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_maxpool_dist_describe.csv',
                       'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_meanpool_dist_describe.csv',
                       'invite_info_[3840, 3860]-[3861, 3867]_quest_hist_minpool_dist_describe.csv',
                      'invite_info_[3840, 3860]-[3861, 3867]_quest_user_follow_topics_cosine_dist_describe.csv',
                          'invite_info_[3840, 3860]-[3861, 3867]_quest_user_follow_topics_euclid_dist_describe.csv',
                      'invite_info_[3840, 3860]-[3861, 3867]_quest_user_interest_topics_cosine_dist_describe.csv',
                      'invite_info_[3840, 3860]-[3861, 3867]_quest_topics_inn_euclid_dist_describe.csv',
                      'invite_info_[3840, 3860]-[3861, 3867]_quest_user_follow_topics_sim_describe.csv',
                    
                    ]
test_invite_files = ['invite_info_test-[3847, 3867]-[3868, 3874].csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_ans_info.csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_invite_describe.csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_topics_itsc_count.csv',
                    'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_topics_dist_describe.csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_maxpool_dist.csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_meanpool_dist.csv',
                      'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_minpool_dist.csv',
                      'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_maxpool_dist_describe.csv',
                      'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_meanpool_dist_describe.csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_hist_minpool_dist_describe.csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_user_follow_topics_cosine_dist_describe.csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_user_follow_topics_euclid_dist_describe.csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_user_interest_topics_cosine_dist_describe.csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_topics_inn_euclid_dist_describe.csv',
                     'invite_info_test-[3847, 3867]-[3868, 3874]_quest_user_follow_topics_sim_describe.csv',
                    
                   ]

old = pd.read_csv(os.path.join(dataDir, 'invite_info_[3840, 3860]-[3861, 3867]_old.csv',), index_col= 0, sep= '\t')

old.drop(columns= ['word_count_mean.1', 'word_count_sum.1',
       'word_count_std.1', 'quest_user_follow_topics_dist_min',
       'quest_user_follow_topics_dist_max',
       'quest_user_follow_topics_dist_mean',
       'quest_user_follow_topics_dist_std', 'quest_topics_inn_dist_max',
       'quest_topics_inn_dist_mean', 'quest_topics_inn_dist_mean.1'], inplace= True)

old['hist_accept_ratio']= old['hist_answer_count'] / old['hist_invite_count']

old['hist_topics_count'] = old['hist_quest_topics'].apply(lambda s: len(s.split(',')) + 1 if isinstance(s, str) else 0)

np.all(old.columns == t.columns)

old.to_csv(os.path.join(dataDir, 'invite_info_[3840, 3860]-[3861, 3867].csv',), sep= '\t')

quest_hist_df.head()

In [13]:
for df in train_invite_df_ls:
    print(df.head().index)

Int64Index([736806, 1266509, 2562710, 4155269, 588392], dtype='int64')
Index(['Q4251023727', 'Q2418257178', 'Q4038278259', 'Q3739876694',
       'Q4120770363'],
      dtype='object', name='question_id')
Int64Index([736806, 1266509, 2562710, 4155269, 588392], dtype='int64')
Int64Index([736806, 1266509, 2562710, 4155269, 588392], dtype='int64')


In [42]:
train_invite_df_ls = []
for file in train_invite_files:
    train_invite_df_ls.append(pd.read_csv(os.path.join(dataDir, file), 
                                          sep= '\t',
                                          # nrows= 1000,
                                          index_col= 0,
                                         na_values = ['-1', -1]))
train_invite_df = pd.concat(train_invite_df_ls, axis= 1)

test_invite_df_ls = []
for file in test_invite_files:
    test_invite_df_ls.append(pd.read_csv(os.path.join(dataDir, file), 
                                          sep= '\t',
                                          # nrows= 1000,
                                          index_col= 0,
                                        na_values = ['-1', -1]))
test_invite_df = pd.concat(test_invite_df_ls, axis= 1)


In [77]:
invite_file_add = 'invite_info_[3840, 3860]-[3861, 3867]_quest_user_interest_topics_sim_describe.csv'
test_invite_file_add =  'invite_info_test-[3847, 3867]-[3868, 3874]_quest_user_interest_topics_sim_describe.csv'
invite_df_add = pd.read_csv(os.path.join(dataDir, invite_file_add), sep= '\t', index_col= 0)

train_invite_df = pd.concat([train_invite_df,invite_df_add], axis= 1)

test_invite_df_add = pd.read_csv(os.path.join(dataDir, test_invite_file_add), sep= '\t', index_col= 0)
test_invite_df = pd.concat([test_invite_df, test_invite_df_add], axis= 1)

In [17]:
print(train_invite_df.shape, test_invite_df.shape)

(2491867, 109) (1141683, 109)


train_invite_df = pd.read_csv(os.path.join(dataDir, train_invite_file), 
                                          sep= '\t',
                                          # nrows= 1000,
                                          index_col= 0)

test_invite_df = pd.read_csv(os.path.join(dataDir, test_invite_file), sep= '\t', 
                             # nrows= 1000, 
                             index_col= 0)

* 还有一些na值没有处理。。。

In [26]:
naCount= train_invite_df.isna().sum()
for n , c in zip(naCount.index, naCount):
    print(n , c)

answer_id 2178656
create_day 0
create_hour 0
create_time 0
create_weekday 0
is_answer 0
is_test 0
question_id 0
user_id 0
user_hist_invite_count 0
user_hist_answer_count 0
user_hist_decline_count 0
user_hist_quest 1041074
user_hist_quest_decline 333304
hist_ans 1041074
user_hist_quest_topics 1041139
user_hist_quest_decline_topics 333354
quest_topics 5173
user_interest_topics 387979
user_follow_topics 327394
last_ans_day 0
days_since_last_ans 0
is_good_mean 1041074
is_good_sum 1041074
is_good_std 1041074
has_picture_mean 1041074
has_picture_sum 1041074
has_picture_std 1041074
has_video_mean 1041074
has_video_sum 1041074
has_video_std 1041074
word_count_mean 1041074
word_count_sum 1041074
word_count_std 1041074
num_zan_mean 1041074
num_zan_sum 1041074
num_zan_std 1041074
num_cancel_zan_mean 1041074
num_cancel_zan_sum 1041074
num_cancel_zan_std 1041074
num_comment_mean 1041074
num_comment_sum 1041074
num_comment_std 1041074
num_collect_mean 1041074
num_collect_sum 1041074
num_collect_std 

In [12]:
print(np.sum(train_invite_df['hist_answer_count'] == 0) / len(train_invite_df), np.sum(test_invite_df['hist_answer_count'] == 0) / len(test_invite_df))


0.4177887503626799 0.44560793144857197


#### 训练集和测试集中各有约 40 % 的人在过去三周没有历史回答记录，只有历史拒绝记录

In [13]:
print(np.sum(train_invite_df['hist_invite_count'] == 0) / len(train_invite_df), np.sum(test_invite_df['hist_invite_count'] == 0) / len(test_invite_df))

0.1047483673887892 0.09972733236809167


In [78]:
print(*train_invite_df.columns, sep= '\n')

answer_id
create_day
create_hour
create_time
create_weekday
is_answer
is_test
question_id
user_id
user_hist_invite_count
user_hist_answer_count
user_hist_decline_count
user_hist_quest
user_hist_quest_decline
hist_ans
user_hist_quest_topics
user_hist_quest_decline_topics
quest_topics
user_interest_topics
user_follow_topics
last_ans_day
days_since_last_ans
is_good_mean
is_good_sum
is_good_std
has_picture_mean
has_picture_sum
has_picture_std
has_video_mean
has_video_sum
has_video_std
word_count_mean
word_count_sum
word_count_std
num_zan_mean
num_zan_sum
num_zan_std
num_cancel_zan_mean
num_cancel_zan_sum
num_cancel_zan_std
num_comment_mean
num_comment_sum
num_comment_std
num_collect_mean
num_collect_sum
num_collect_std
num_thanks_mean
num_thanks_sum
num_thanks_std
num_report_mean
num_report_sum
num_report_std
num_useless_mean
num_useless_sum
num_useless_std
num_oppose_mean
num_oppose_sum
num_oppose_std
word_count_mean.1
word_count_sum.1
word_count_std.1
quest_hist_is_good_mean
quest_hist_i

In [80]:
quest_feat_dict = {'sparse': ['has_describe', ],
                   'dense': [
                            'describe_W_length',
                             'title_W_length',
                             'question_topics_length',
                             'question_invite_count',
                           'question_create_day',
                             ]
                   }

user_feat_dict = {'sparse': [
    'gender',
    'visit_freq',
    'binary_A',
    'binary_B',
    'binary_C',
    'binary_D',
    'binary_E',
    'category_A',
    'category_B',
    'category_C',
    'category_D',
    'category_E',
],
    'dense': [
        'user_invite_count',
        'salt_value',
        'gender_count', 'visit_freq_count', 'binary_A_count', 'binary_B_count',
       'binary_C_count', 'binary_D_count', 'binary_E_count',
       'category_A_count', 'category_B_count', 'category_C_count',
       'category_D_count', 'category_E_count'
        # 'follow_topics_mp',
        # 'interest_topics_wp',
    ]
}

context_feat_dict = {
    'sparse': [
        'create_hour',
        'create_weekday',
    ],
    'dense': [ 
        'create_day',
        'user_hist_invite_count',
        'user_hist_answer_count',
        'user_hist_decline_count',
        'quest_user_hist_topics_itsc_count',
        'quest_user_hist_decline_topics_itsc_count',
        'quest_user_follow_topics_itsc_count',
        'quest_user_interest_topics_itsc_weight',
        'quest_user_interest_topics_itsc_count',
       'days_since_last_ans',  
        'is_good_mean',
       'is_good_sum', 'is_good_std', 'has_picture_mean', 'has_picture_sum',
       'has_picture_std', 'has_video_mean', 'has_video_sum', 'has_video_std',
       'word_count_mean', 'word_count_sum', 'word_count_std', 'num_zan_mean',
       'num_zan_sum', 'num_zan_std', 'num_cancel_zan_mean',
       'num_cancel_zan_sum', 'num_cancel_zan_std', 'num_comment_mean',
       'num_comment_sum', 'num_comment_std', 'num_collect_mean',
       'num_collect_sum', 'num_collect_std', 'num_thanks_mean',
       'num_thanks_sum', 'num_thanks_std', 'num_report_mean', 'num_report_sum',
       'num_report_std', 'num_useless_mean', 'num_useless_sum',
       'num_useless_std', 'num_oppose_mean', 'num_oppose_sum',
       'num_oppose_std', 
        # 'hist_accept_ratio',
        'quest_hist_ans_count',
       'quest_hist_invite_count', 
        'quest_hist_decline_count',
        'quest_hist_is_good_mean',
        'quest_hist_is_good_sum',
        'quest_hist_is_good_std',
        'quest_hist_has_picture_mean',
        'quest_hist_has_picture_sum',
        'quest_hist_has_picture_std',
        'quest_hist_has_video_mean',
        'quest_hist_has_video_sum',
        'quest_hist_has_video_std',
        'quest_hist_word_count_mean',
        'quest_hist_word_count_sum',
        'quest_hist_word_count_std',
        'quest_hist_num_zan_mean',
        'quest_hist_num_zan_sum',
        'quest_hist_num_zan_std',
        'quest_hist_num_cancel_zan_mean',
        'quest_hist_num_cancel_zan_sum',
        'quest_hist_num_cancel_zan_std',
        'quest_hist_num_comment_mean',
        'quest_hist_num_comment_sum',
        'quest_hist_num_comment_std',
        'quest_hist_num_collect_mean',
        'quest_hist_num_collect_sum',
        'quest_hist_num_collect_std',
        'quest_hist_num_thanks_mean',
        'quest_hist_num_thanks_sum',
        'quest_hist_num_thanks_std',
        'quest_hist_num_report_mean',
        'quest_hist_num_report_sum',
        'quest_hist_num_report_std',
        'quest_hist_num_useless_mean',
        'quest_hist_num_useless_sum',
        'quest_hist_num_useless_std',
        'quest_hist_num_oppose_mean',
        'quest_hist_num_oppose_sum',
        'quest_hist_num_oppose_std',
        'quest_user_follow_topics_euclid_dist_min',
       'quest_user_follow_topics_euclid_dist_max',
       'quest_user_follow_topics_euclid_dist_mean',
       'quest_user_follow_topics_euclid_dist_std',
       'quest_topics_inn_euclid_dist_max',
       'quest_topics_inn_euclid_dist_mean',
       'quest_topics_inn_euclid_dist_std',
       'quest_user_hist_topics_euclid_dist_min',
       'quest_user_hist_topics_euclid_dist_max',
       'quest_user_hist_topics_euclid_dist_std',
       'quest_user_hist_topics_euclid_dist_mean',
        'quest_hist_maxpool_cosine_dist_min',
        'quest_hist_maxpool_cosine_dist_max',
        'quest_hist_maxpool_cosine_dist_mean',
        'quest_hist_maxpool_cosine_dist_std',
        'quest_hist_maxpool_euclid_dist_min',
        'quest_hist_maxpool_euclid_dist_max',
        'quest_hist_maxpool_euclid_dist_mean',
        'quest_hist_maxpool_euclid_dist_std',
        'quest_hist_meanpool_cosine_dist_min',
        'quest_hist_meanpool_cosine_dist_max',
        'quest_hist_meanpool_cosine_dist_mean',
        'quest_hist_meanpool_cosine_dist_std',
        'quest_hist_meanpool_euclid_dist_min',
        'quest_hist_meanpool_euclid_dist_max',
        'quest_hist_meanpool_euclid_dist_mean',
        'quest_hist_meanpool_euclid_dist_std',
        'quest_hist_minpool_cosine_dist_min',
        'quest_hist_minpool_cosine_dist_max',
        'quest_hist_minpool_cosine_dist_mean',
        'quest_hist_minpool_cosine_dist_std',
        'quest_hist_minpool_euclid_dist_min',
        'quest_hist_minpool_euclid_dist_max',
        'quest_hist_minpool_euclid_dist_mean',
        'quest_hist_minpool_euclid_dist_std',
        'hist_topics_count',
        'quest_user_follow_topics_cosine_dist_mean',
        'quest_user_follow_topics_cosine_dist_std',
        'quest_user_follow_topics_cosine_dist_top3_mean',
        'quest_user_interest_topics_cosine_weight_min_dist',
        'quest_user_interest_topics_cosine_weight_max_dist',
        'quest_user_interest_topics_cosine_weight_mean_dist',
        'quest_user_interest_topics_cosine_min_dist_top3_weight_sum',
        'quest_user_interest_topics_cosine_min_dist_tail3_weight_sum',
        'quest_user_interest_topics_cosine_min_dist_top_weight',
        'quest_user_interest_topics_cosine_min_dist_tail_weight',
        'quest_user_follow_topics_max_sim_min',
        'quest_user_follow_topics_max_sim_max',
        'quest_user_follow_topics_max_sim_std',
        'quest_user_follow_topics_max_sim_mean',
        'quest_user_follow_topics_max_sim_top3_mean',
        'quest_user_follow_topics_max_sim_tail3_mean',
        'quest_user_interest_topics_rsum_sim_min',
        'quest_user_interest_topics_rsum_sim_max',
        'quest_user_interest_topics_rsum_sim_std',
        'quest_user_interest_topics_rsum_sim_mean',
        'quest_user_interest_topics_rsum_sim_top3_mean',
        'quest_user_interest_topics_rsum_sim_tail3_mean',
        'quest_user_interest_topics_csum_sim_top3_weight',
        'quest_user_interest_topics_csum_sim_tail3_weight',
        'quest_user_interest_topics_csum_sim_top_weight',
        'quest_user_interest_topics_csum_sim_tail_weight',
    ]
}


In [81]:
context_feat_names= ['question_id', 'user_id'] + context_feat_dict['sparse'] + context_feat_dict['dense']
quest_feat_names = quest_feat_dict['sparse'] + quest_feat_dict['dense']
user_feat_names = user_feat_dict['sparse'] + user_feat_dict['dense']


In [82]:
quest_feats = quest_df[quest_feat_names]
context_feats = train_invite_df[context_feat_names]
user_feats = user_df[user_feat_names]
label = train_invite_df['is_answer']


part_df = pd.merge(context_feats, quest_feats, left_on= 'question_id', right_index=True,  how= 'left', validate='many_to_one')
data = pd.merge(part_df, user_feats, left_on= 'user_id', right_index= True, how= 'left', validate= 'many_to_one')


assert len(data) == len(train_invite_df)

data['days_since_quest_create'] = data['create_day'] -data['question_create_day']



In [83]:
data.head()

,question_id,user_id,create_hour,create_weekday,create_day,user_hist_invite_count,user_hist_answer_count,user_hist_decline_count,quest_user_hist_topics_itsc_count,quest_user_hist_decline_topics_itsc_count,...,binary_B_count,binary_C_count,binary_D_count,binary_E_count,category_A_count,category_B_count,category_C_count,category_D_count,category_E_count,days_since_quest_create
736806,Q4251023727,M1000000382,17,4,3861,4.0,1.0,3.0,0,1,...,602125,1874610,1646667,1826951,62180,409524,90625,90625,1767878,5
1266509,Q2418257178,M1000000382,20,5,3862,4.0,1.0,3.0,0,1,...,602125,1874610,1646667,1826951,62180,409524,90625,90625,1767878,0
2562710,Q4038278259,M1000000382,18,1,3865,4.0,1.0,3.0,0,1,...,602125,1874610,1646667,1826951,62180,409524,90625,90625,1767878,0
4155269,Q3739876694,M1000000382,21,2,3866,4.0,1.0,3.0,0,1,...,602125,1874610,1646667,1826951,62180,409524,90625,90625,1767878,6
588392,Q4120770363,M1000000983,18,1,3865,13.0,3.0,10.0,0,0,...,1329529,57044,1646667,1826951,7374,91151,44031,778428,163776,4


In [84]:
data.columns

Index(['question_id', 'user_id', 'create_hour', 'create_weekday', 'create_day',
       'user_hist_invite_count', 'user_hist_answer_count',
       'user_hist_decline_count', 'quest_user_hist_topics_itsc_count',
       'quest_user_hist_decline_topics_itsc_count',
       ...
       'binary_B_count', 'binary_C_count', 'binary_D_count', 'binary_E_count',
       'category_A_count', 'category_B_count', 'category_C_count',
       'category_D_count', 'category_E_count', 'days_since_quest_create'],
      dtype='object', length=184)

In [85]:
sparse_feat_names = context_feat_dict['sparse'] + user_feat_dict['sparse'] + quest_feat_dict['sparse']

df2use = data.drop(columns = ['question_id', 'user_id', 'create_day', 'question_create_day'])

In [86]:
from lightgbm import early_stopping

In [54]:
model_lgb = LGBMClassifier(boosting_type='gbdt', num_leaves=64, learning_rate=0.05, n_estimators=4000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                         min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree= 1, seed=1000, n_jobs=32, reg_alpha= 3, reg_lambda= 5, silent=True, 
                           # is_unbalance= True
                          )

In [55]:
from sklearn.model_selection import StratifiedKFold

In [56]:
skf = StratifiedKFold(n_splits= 3)

In [87]:
for train_idx, val_idx in skf.split(df2use, label):
    model_lgb.fit(df2use.iloc[train_idx], label.iloc[train_idx],
                  eval_set= [(df2use.iloc[val_idx], label.iloc[val_idx]), ],
                  eval_names= ['validation', ],
                  eval_metric= ['auc', 'logloss'],
                 categorical_feature = sparse_feat_names,
                  early_stopping_rounds= 5,
                 callbacks= [early_stopping(5, first_metric_only= True), ]
                 )


/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/lightgbm/basic.py:1247: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['binary_A', 'binary_B', 'binary_C', 'binary_D', 'binary_E', 'category_A', 'category_B', 'category_C', 'category_D', 'category_E', 'create_hour', 'create_weekday', 'gender', 'has_describe', 'visit_freq']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[1]	validation's auc: 0.78409	validation's binary_logloss: 0.370828
Training until validation scores don't improve for 5 rounds
Training until validation scores don't improve for 5 rounds
[2]	validation's auc: 0.791013	validation's binary_logloss: 0.364663
[3]	validation's auc: 0.794928	validation's binary_logloss: 0.359307
[4]	validation's auc: 0.796857	validation's binary_logloss: 0.354587
[5]	validation's auc: 0.797668	validation's binary_logloss: 0.350403
[6]	validation's auc: 0.799176	validation's binary_logloss: 0.346663
[7]	validation's auc: 0.800938	validation's binary_logloss: 0.343264
[8]	validation's auc: 0.802051	validation's binary_logloss: 0.340186
[9]	validation's auc: 0.8029	validation's binary_logloss: 0.337373
[10]	validation's auc: 0.803717	validation's binary_logloss: 0.334796
[11]	validation's auc: 0.804863	validation's binary_logloss: 0.332363
[12]	validation's auc: 0.806072	validation's binary_logloss: 0.33014
[13]	validation's auc: 0.806963	validation's binary_l

[117]	validation's auc: 0.835032	validation's binary_logloss: 0.288717
[118]	validation's auc: 0.835055	validation's binary_logloss: 0.288687
[119]	validation's auc: 0.83509	validation's binary_logloss: 0.288657
[120]	validation's auc: 0.835158	validation's binary_logloss: 0.288607
[121]	validation's auc: 0.835216	validation's binary_logloss: 0.288562
[122]	validation's auc: 0.83528	validation's binary_logloss: 0.28851
[123]	validation's auc: 0.835314	validation's binary_logloss: 0.288481
[124]	validation's auc: 0.835415	validation's binary_logloss: 0.288402
[125]	validation's auc: 0.835541	validation's binary_logloss: 0.28831
[126]	validation's auc: 0.835676	validation's binary_logloss: 0.288218
[127]	validation's auc: 0.835759	validation's binary_logloss: 0.288148
[128]	validation's auc: 0.835827	validation's binary_logloss: 0.288099
[129]	validation's auc: 0.835909	validation's binary_logloss: 0.288045
[130]	validation's auc: 0.835953	validation's binary_logloss: 0.288012
[131]	vali

[233]	validation's auc: 0.839898	validation's binary_logloss: 0.284886
[234]	validation's auc: 0.839901	validation's binary_logloss: 0.284881
[235]	validation's auc: 0.839902	validation's binary_logloss: 0.284879
[236]	validation's auc: 0.839914	validation's binary_logloss: 0.284868
[237]	validation's auc: 0.839919	validation's binary_logloss: 0.284864
[238]	validation's auc: 0.839938	validation's binary_logloss: 0.284849
[239]	validation's auc: 0.839982	validation's binary_logloss: 0.284814
[240]	validation's auc: 0.840003	validation's binary_logloss: 0.284792
[241]	validation's auc: 0.840016	validation's binary_logloss: 0.28478
[242]	validation's auc: 0.840038	validation's binary_logloss: 0.284765
[243]	validation's auc: 0.840072	validation's binary_logloss: 0.284746
[244]	validation's auc: 0.840097	validation's binary_logloss: 0.284727
[245]	validation's auc: 0.840114	validation's binary_logloss: 0.284715
[246]	validation's auc: 0.840157	validation's binary_logloss: 0.284684
[247]	v

[2]	validation's auc: 0.795265	validation's binary_logloss: 0.364526
[3]	validation's auc: 0.797631	validation's binary_logloss: 0.359127
[4]	validation's auc: 0.799724	validation's binary_logloss: 0.354396
[5]	validation's auc: 0.800602	validation's binary_logloss: 0.350195
[6]	validation's auc: 0.801983	validation's binary_logloss: 0.346387
[7]	validation's auc: 0.80353	validation's binary_logloss: 0.342919
[8]	validation's auc: 0.80432	validation's binary_logloss: 0.339805
[9]	validation's auc: 0.805546	validation's binary_logloss: 0.336936
[10]	validation's auc: 0.806895	validation's binary_logloss: 0.334283
[11]	validation's auc: 0.808096	validation's binary_logloss: 0.331799
[12]	validation's auc: 0.808843	validation's binary_logloss: 0.329568
[13]	validation's auc: 0.809724	validation's binary_logloss: 0.327423
[14]	validation's auc: 0.810466	validation's binary_logloss: 0.32547
[15]	validation's auc: 0.811256	validation's binary_logloss: 0.323625
[16]	validation's auc: 0.811859

[120]	validation's auc: 0.83687	validation's binary_logloss: 0.287166
[121]	validation's auc: 0.836968	validation's binary_logloss: 0.287067
[122]	validation's auc: 0.837087	validation's binary_logloss: 0.286966
[123]	validation's auc: 0.837175	validation's binary_logloss: 0.286903
[124]	validation's auc: 0.83719	validation's binary_logloss: 0.286878
[125]	validation's auc: 0.83726	validation's binary_logloss: 0.286823
[126]	validation's auc: 0.837286	validation's binary_logloss: 0.286786
[127]	validation's auc: 0.837311	validation's binary_logloss: 0.286759
[128]	validation's auc: 0.837352	validation's binary_logloss: 0.286722
[129]	validation's auc: 0.837389	validation's binary_logloss: 0.286692
[130]	validation's auc: 0.837495	validation's binary_logloss: 0.286592
[131]	validation's auc: 0.837508	validation's binary_logloss: 0.286576
[132]	validation's auc: 0.837524	validation's binary_logloss: 0.286562
[133]	validation's auc: 0.837555	validation's binary_logloss: 0.286528
[134]	val

[236]	validation's auc: 0.841159	validation's binary_logloss: 0.283552
[237]	validation's auc: 0.84116	validation's binary_logloss: 0.28355
[238]	validation's auc: 0.841164	validation's binary_logloss: 0.283547
[239]	validation's auc: 0.84119	validation's binary_logloss: 0.283527
[240]	validation's auc: 0.841201	validation's binary_logloss: 0.283521
[241]	validation's auc: 0.841223	validation's binary_logloss: 0.283511
[242]	validation's auc: 0.841234	validation's binary_logloss: 0.2835
[243]	validation's auc: 0.841247	validation's binary_logloss: 0.283491
[244]	validation's auc: 0.84128	validation's binary_logloss: 0.28347
[245]	validation's auc: 0.841291	validation's binary_logloss: 0.283454
[246]	validation's auc: 0.841312	validation's binary_logloss: 0.283436
[247]	validation's auc: 0.841334	validation's binary_logloss: 0.283421
[248]	validation's auc: 0.841334	validation's binary_logloss: 0.283422
[249]	validation's auc: 0.84135	validation's binary_logloss: 0.283409
[250]	validati

[16]	validation's auc: 0.811278	validation's binary_logloss: 0.322364
[17]	validation's auc: 0.811987	validation's binary_logloss: 0.320758
[18]	validation's auc: 0.812641	validation's binary_logloss: 0.31925
[19]	validation's auc: 0.813525	validation's binary_logloss: 0.317806
[20]	validation's auc: 0.81417	validation's binary_logloss: 0.316455
[21]	validation's auc: 0.814575	validation's binary_logloss: 0.315241
[22]	validation's auc: 0.815314	validation's binary_logloss: 0.314051
[23]	validation's auc: 0.8159	validation's binary_logloss: 0.31292
[24]	validation's auc: 0.81647	validation's binary_logloss: 0.311858
[25]	validation's auc: 0.817011	validation's binary_logloss: 0.310878
[26]	validation's auc: 0.81751	validation's binary_logloss: 0.309923
[27]	validation's auc: 0.818047	validation's binary_logloss: 0.309043
[28]	validation's auc: 0.818563	validation's binary_logloss: 0.308196
[29]	validation's auc: 0.819084	validation's binary_logloss: 0.307391
[30]	validation's auc: 0.81

[134]	validation's auc: 0.837353	validation's binary_logloss: 0.287043
[135]	validation's auc: 0.837459	validation's binary_logloss: 0.286962
[136]	validation's auc: 0.837564	validation's binary_logloss: 0.286864
[137]	validation's auc: 0.837586	validation's binary_logloss: 0.286842
[138]	validation's auc: 0.837596	validation's binary_logloss: 0.286823
[139]	validation's auc: 0.837698	validation's binary_logloss: 0.286738
[140]	validation's auc: 0.837738	validation's binary_logloss: 0.286705
[141]	validation's auc: 0.837778	validation's binary_logloss: 0.286668
[142]	validation's auc: 0.837901	validation's binary_logloss: 0.286583
[143]	validation's auc: 0.837927	validation's binary_logloss: 0.286558
[144]	validation's auc: 0.838007	validation's binary_logloss: 0.286491
[145]	validation's auc: 0.83804	validation's binary_logloss: 0.286467
[146]	validation's auc: 0.838093	validation's binary_logloss: 0.286429
[147]	validation's auc: 0.838115	validation's binary_logloss: 0.286404
[148]	v

[250]	validation's auc: 0.841142	validation's binary_logloss: 0.28395
[251]	validation's auc: 0.841155	validation's binary_logloss: 0.283942
[252]	validation's auc: 0.841194	validation's binary_logloss: 0.283913
[253]	validation's auc: 0.841205	validation's binary_logloss: 0.283901
[254]	validation's auc: 0.841214	validation's binary_logloss: 0.283895
[255]	validation's auc: 0.841255	validation's binary_logloss: 0.283864
[256]	validation's auc: 0.84125	validation's binary_logloss: 0.283869
[257]	validation's auc: 0.841256	validation's binary_logloss: 0.283863
[258]	validation's auc: 0.841285	validation's binary_logloss: 0.283839
[259]	validation's auc: 0.84131	validation's binary_logloss: 0.283823
[260]	validation's auc: 0.841321	validation's binary_logloss: 0.283816
[261]	validation's auc: 0.841341	validation's binary_logloss: 0.283801
[262]	validation's auc: 0.841391	validation's binary_logloss: 0.283758
[263]	validation's auc: 0.84143	validation's binary_logloss: 0.283724
[264]	vali

* 1209 使用除距离属性外的所有属性，线下84.1,线上80.16
* 1209 加入距离属性，丢掉 question_create_day，线下84.1, 线上80.13
* 1212 去除所有由embedding计算的距离特征，包括'quest_user_follow_topics_min_distance','quest_user_interest_topics_min_distance',学习率0.01, 400轮，线下82.8
* 1212 加上 'quest_user_follow_topics_min_distance','quest_user_interest_topics_min_distance',学习率0.01, 400轮，线下82.9
* 1213 重新计算所有属性，不使用embeding计算得到的距离属性，线下 0.835
* 1213 加入embedding 计算得到距离属性，线下0.8418，线下0.806
* 1214 加入topic 相似度得到quest user_follow_topics话题相似度特征，线下0.842, 
* 1214 加入topic相似度得到的quest user_interest_topics话题相似度特征,线下84.12

#### 去除属性

In [123]:
df2use_2 = df2use.drop(columns = ['quest_user_follow_topics_min_distance',
       'quest_user_interest_topics_min_distance', 'quest_user_follow_topics_dist_min',
       'quest_user_follow_topics_dist_max',
       'quest_user_follow_topics_dist_mean',
       'quest_user_follow_topics_dist_std', 'quest_topics_inn_dist_max',
       'quest_topics_inn_dist_mean', 'quest_topics_inn_dist_mean.1',])

KeyError: "['quest_user_follow_topics_dist_min' 'quest_user_follow_topics_dist_max'\n 'quest_user_follow_topics_dist_mean' 'quest_user_follow_topics_dist_std'\n 'quest_topics_inn_dist_max' 'quest_topics_inn_dist_mean'\n 'quest_topics_inn_dist_mean.1'] not found in axis"

In [124]:
from random import shuffle

In [164]:
val_ratio= 0.33
n_samples = len(df2use)
idx = list(range(n_samples))
shuffle(idx)
train_idx = idx[:int(n_samples * val_ratio)]
val_idx = idx[int(n_samples * val_ratio) + 1:]



In [128]:
model_lgb_2 = LGBMClassifier(boosting_type='gbdt', num_leaves=64, learning_rate=0.1, n_estimators=400,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree= 1, seed=1000, n_jobs=32, reg_alpha=3, reg_lambda=5, silent=True, 
                           # is_unbalance= True
                          )

In [165]:
model_lgb_2.fit(df2use_2.iloc[train_idx], label.iloc[train_idx],
              eval_set= [(df2use_2.iloc[val_idx], label.iloc[val_idx]), ],
              eval_names= ['validation', ],
              eval_metric= ['auc', 'logloss'],
             categorical_feature = sparse_feat_names,
              early_stopping_rounds= 5,
             callbacks= [early_stopping(5, first_metric_only= True), ]
             )


[1]	validation's auc: 0.769871	validation's binary_logloss: 0.447026
Training until validation scores don't improve for 5 rounds
Training until validation scores don't improve for 5 rounds
[2]	validation's auc: 0.780978	validation's binary_logloss: 0.435306
[3]	validation's auc: 0.784583	validation's binary_logloss: 0.426072
[4]	validation's auc: 0.787985	validation's binary_logloss: 0.418331
[5]	validation's auc: 0.789721	validation's binary_logloss: 0.411956
[6]	validation's auc: 0.791697	validation's binary_logloss: 0.406436
[7]	validation's auc: 0.79375	validation's binary_logloss: 0.401791
[8]	validation's auc: 0.795305	validation's binary_logloss: 0.39763
[9]	validation's auc: 0.796509	validation's binary_logloss: 0.39412
[10]	validation's auc: 0.797861	validation's binary_logloss: 0.390937
[11]	validation's auc: 0.798805	validation's binary_logloss: 0.388241
[12]	validation's auc: 0.800115	validation's binary_logloss: 0.385592
[13]	validation's auc: 0.800904	validation's binary_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1,
        importance_type='split', learning_rate=0.1, max_bin=425,
        max_depth=-1, min_child_samples=10, min_child_weight=5,
        min_split_gain=0, n_estimators=400, n_jobs=32, num_leaves=64,
        objective='binary', random_state=None, reg_alpha=3, reg_lambda=5,
        seed=1000, silent=True, subsample=0.8, subsample_for_bin=50000,
        subsample_freq=1)

### 问题和用户的距离属性auc 提高了0.1

### 测试

In [88]:
test_context_feats = test_invite_df[context_feat_names]



test_part_df = pd.merge(test_context_feats, quest_feats, left_on= 'question_id', right_index=True,  how= 'left', validate='many_to_one')
test_data = pd.merge(test_part_df, user_feats, left_on= 'user_id', right_index= True, how= 'left', validate= 'many_to_one')

assert len(test_data) == len(test_invite_df)

test_data['days_since_quest_create'] = test_data['create_day'] - test_data['question_create_day']

test_df2use = test_data.drop(columns = ['question_id', 'user_id', 'create_day','question_create_day'])

In [89]:
pred_values = model_lgb.predict_proba(test_df2use)

#### 取第二列

In [90]:
len(test_df2use)

1141683

In [91]:
configStr= 'lgbm_1214'
res_df = test_invite_df[['question_id', 'user_id', 'create_time']]
assert len(pred_values) == len(res_df)
res_df['predict_value'] = pred_values[:, 1]

res_df.to_csv(os.path.join(dataDir, 'result_%s.txt' %(configStr, )), sep= '\t', header= False, index= False,
               columns = ['question_id', 'user_id', 'create_time', 'predict_value'])

/home/sc3/miniconda2/envs/ydzhang/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
